# Main class - `EEGTest.m`

In [ ]:
%%file EEGTest.m
classdef EEGTest < EEGParameters & EEGReconstruction
    methods
        function obj = EEGTest()
            % % Constructor of EEGTest class
        end
        function obj = testsetup(obj)
            % Test for equality of SETUPs.

            % From new code
            parameters = obj.generate();
            obj = obj.init();
            obj = obj.setparameters(parameters(1));
            
            % From original code
            SETUP = testprelude();
            
            % Test
            [common, d1, d2] = comp_struct(obj.SETUP, SETUP);
            % Fields sel_msh, sel_geo_deep_thalami, sel_geo_deep_icosahedron642,
            % sel_atl, sel_vol, sel_ele, sel_src, thalamus, DEBUG, SINR,
            % rngMesNoise, rngBcgActiv, rngIntActiv, DATE, NAME, totSimCount,
            % rngSimCount, ActivityInd were added to EEGSImulation so that
            % there may not be in SETUP and everything is OK.
            
            extrafields = {'thalamus', 'DEBUG', 'WARNINGS', 'SINR', 'DATE', 'NAME', 'PATH', 'SINR_RNG', 'SBNR_RNG', 'SMNR_RNG', 'pPNT' };
            if isempty(setdiff(fieldnames(d1), extrafields)) && isempty(d2)
                disp('>>>   TEST FOR SETUP PREPARATION DONE PROPERLY!');
            else
                error('<<<   ERROR DURING TESTING SETUP PREPARATION!');
            end
        end
        function obj = testsignals(obj)
            % Test for equality of generated signals.

            % From new code
            obj = obj.setsignals();
            
            % From original code
            temp = testtimeseries();

            [common, d1, d2] = comp_struct(obj.MODEL, temp);
            
            if isempty(d1) && isempty(d2)
                disp(">>>   TEST FOR SIGNAL GENERATION DONE PROPERLY!");
            else
                error('<<<   ERROR DURING TESTING SIGNAL GENERATION!');
            end
        end
        function obj = testleadfields(obj)
            % Test for equality of lead-fields.

            % From new code
            obj = obj.setleadfields();
            
            % From original code
            temp = testgeometry();
            
            %[common, d1, d2] = comp_struct(obj.MODEL, temp);
            
            model_test = obj.MODEL;
            temp_test = temp;
            
            % We have to remove all fields, which are inherently
            % differet, i.e. proctime, pwd etc.
            model_test.sim_lfg_SrcActiv_orig.lfg.cfg = rmfield(model_test.sim_lfg_SrcActiv_orig.lfg.cfg, 'callinfo');
            temp_test.sim_lfg_SrcActiv_orig.lfg.cfg = rmfield(temp_test.sim_lfg_SrcActiv_orig.lfg.cfg, 'callinfo');
            model_test.sim_lfg_SrcActiv_pert.lfg.cfg = rmfield(model_test.sim_lfg_SrcActiv_pert.lfg.cfg, 'callinfo');
            temp_test.sim_lfg_SrcActiv_pert.lfg.cfg = rmfield(temp_test.sim_lfg_SrcActiv_pert.lfg.cfg, 'callinfo');
            
            model_test.sim_lfg_IntActiv_orig.lfg.cfg = rmfield(model_test.sim_lfg_IntActiv_orig.lfg.cfg, 'callinfo');
            temp_test.sim_lfg_IntActiv_orig.lfg.cfg = rmfield(temp_test.sim_lfg_IntActiv_orig.lfg.cfg, 'callinfo');
            model_test.sim_lfg_IntActiv_pert.lfg.cfg = rmfield(model_test.sim_lfg_IntActiv_pert.lfg.cfg, 'callinfo');
            temp_test.sim_lfg_IntActiv_pert.lfg.cfg = rmfield(temp_test.sim_lfg_IntActiv_pert.lfg.cfg, 'callinfo');
            
            model_test.sim_lfg_BcgActiv_orig.lfg.cfg = rmfield(model_test.sim_lfg_BcgActiv_orig.lfg.cfg, 'callinfo');
            temp_test.sim_lfg_BcgActiv_orig.lfg.cfg = rmfield(temp_test.sim_lfg_BcgActiv_orig.lfg.cfg, 'callinfo');
            model_test.sim_lfg_BcgActiv_pert.lfg.cfg = rmfield(model_test.sim_lfg_BcgActiv_pert.lfg.cfg, 'callinfo');
            temp_test.sim_lfg_BcgActiv_pert.lfg.cfg = rmfield(temp_test.sim_lfg_BcgActiv_pert.lfg.cfg, 'callinfo');
            
            %[common, d1, d2] = comp_struct(obj.MODEL.sim_lfg_SrcActiv_orig, temp.sim_lfg_SrcActiv_orig);
            [common, d1, d2] = comp_struct(model_test.sim_lfg_SrcActiv_orig.lfg.cfg, temp_test.sim_lfg_SrcActiv_orig.lfg.cfg);      
            
            if isempty(d1) && isempty(d2)
                disp(">>>   TEST FOR LEADFIELD COMPUTATION DONE PROPERLY!");
            else
                error('<<<   ERROR DURING TESTING SETUP PREPARATION!');
            end
        end
        function obj = testfilters(obj)
            % Test for equality of calculated filters.

            % From new code
            filters = [ "LCMV", "MMSE", "ZF", "RANDN", "ZEROS", "EIG-LCMV", ...
            "sMVP_MSE", "sMVP_R", "sMVP_N", "sMVP_NL_MSE", ...
            "sMVP_NL_R", "sMVP_NL_N", "NL" ];
            obj = obj.setpreparations();
            obj = obj.setfilters(filters);
            model_test = [];
            model_test.rec_opt = obj.MODEL.rec_opt;
            model_test.rec_flt = obj.MODEL.rec_flt;
            model_test.rec_sig = obj.MODEL.rec_sig;
            
            % From original code
            temp_test = testspatialfiltering();
            
            [common, d1, d2] = comp_struct(model_test, temp_test);
                        
            if isempty(d1) && isempty(d2)
                disp(">>>   TEST FOR FILTERS DONE PROPERLY!");
            else
                error('<<<   ERROR DURING TESTING FILTERS!');
            end
        end
        function obj = testerrors(obj)
            % Test for equality of calculated errors.

            load("tangled.mat");

            model_table = obj.MODEL.rec_res.table;
            temp_table = rec_res.table;

            if (isequal(sortrows(model_table), sortrows(temp_table)))
                disp(">>>   TEST FOR ERRORS DONE PROPERLY!");
            else
                error('<<<   ERROR DURING TESTING ERRORS!');
            end 
        end 
    end
end

# Auxiliary functions

In [ ]:
%%file testprelude.m
function SETUP = testprelude()
    % Importing code generated from Org-mode files.

    run('./tangled/tg_s07_BATCH___01_Prelude.m');
    save("tangled.mat"); % Locally save all variable names and values
end

In [ ]:
%%file testtimeseries.m
function temp = testtimeseries()
    % Importing code generated from Org-mode files.

    load("tangled.mat");
    SETUP.DEBUG = 0; % Added for simplicity

    run('./tangled/tg_s01_Timeseries___01_SrcActiv.m');
    run('./tangled/tg_s01_Timeseries___02_IntActiv.m');
    run('./tangled/tg_s01_Timeseries___03_BcgActiv.m');
    run('./tangled/tg_s01_Timeseries___04_MesNoise.m');

    temp.sim_sig_SrcActiv = sim_sig_SrcActiv;
    temp.sim_sig_IntActiv = sim_sig_IntActiv;
    temp.sim_sig_BcgActiv = sim_sig_BcgActiv;
    temp.sim_sig_MesNoise = sim_sig_MesNoise;
    
    save("tangled.mat");
end

In [ ]:
%%file testgeometry.m
function temp = testgeometry()
    % Importing code generated from Org-mode files.

    load("tangled.mat");

    run('./tangled/tg_s02_Geometry___01_RandSamp.m');
    run('./tangled/tg_s02_Geometry___02_Indices.m');
    run('./tangled/tg_s02_Geometry___03_Coordinates.m');
    run('./tangled/tg_s02_Geometry___04_DeepSrc.m');
    run('./tangled/tg_s02_Geometry___05_Perturbation.m');
    run('./tangled/tg_s02_Geometry___06_lfg.m');
    run('./tangled/tg_s03_Forward_modeling.m');

    temp.sim_sig_SrcActiv = sim_sig_SrcActiv;
    temp.sim_sig_IntActiv = sim_sig_IntActiv;
    temp.sim_sig_BcgActiv = sim_sig_BcgActiv;
    temp.sim_sig_MesNoise = sim_sig_MesNoise;

    temp.sim_geo_cort           = sim_geo_cort;
    temp.sim_geo_deep           = sim_geo_deep;
    temp.sim_geo                = sim_geo;
    temp.sim_lfg_SrcActiv_orig  = sim_lfg_SrcActiv_orig;
    temp.sim_lfg_SrcActiv_pert  = sim_lfg_SrcActiv_pert;
    temp.sim_lfg_IntActiv_orig  = sim_lfg_IntActiv_orig;
    temp.sim_lfg_IntActiv_pert  = sim_lfg_IntActiv_pert;
    temp.sim_lfg_BcgActiv_orig  = sim_lfg_BcgActiv_orig;
    temp.sim_lfg_BcgActiv_pert  = sim_lfg_BcgActiv_pert;
    
    save("tangled.mat");
end

In [ ]:
%%file testspatialfiltering.m
function temp = testspatialfiltering()
    % Importing code generated from Org-mode files.

    load("tangled.mat");

    run('./tangled/tg_s04_Preparation___00_SNRs_Adjustment.m');
    run('./tangled/tg_s04_Preparation___01_Signal_Covariances.m');
    run('./tangled/tg_s05_Spatial_filtering___01_Preparation.m');
    run('./tangled/tg_s05_Spatial_filtering___02_Definitions.m');
    run('./tangled/tg_s05_Spatial_filtering___03_Execution.m');
    run('./tangled/tg_s06_Error_evaluation.m');
    
    clear temp;
    
    temp.rec_opt = rec_opt;
    temp.rec_flt = rec_flt;
    temp.rec_sig = rec_sig;
    
    save("tangled.mat");
end